<a href="https://colab.research.google.com/github/Toadoum/American-Sign-Language-detection-using-CNN/blob/main/Sakayo_Toadoum_Sari_LogRegression_students_version_OK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

# Logistic Regression Implementation

# Instructions:
1. Save a copy of this notebook in your drive;
2. Name it by: `Your_fullname_LogRegression.ipynb`
3. Close the original notebook
4. Start typing your codes.

`Note:` Discuss with your neighbors.

# Part 1: Data preparation

## Import Dataset Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Question 0: Import the dataset Dataset

In [3]:
##### WRITE YOUR CODE HERE #####

data= pd.read_csv('/content/diabetes.csv')
#### END CODE ####

Question 1: Print the first five rows of the Dataset

In [4]:
##### WRITE YOUR CODE HERE #####
data.head(5)
#### END CODE ####

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Question 2: Check the types of your variables/features

In [5]:
##### WRITE YOUR CODE HERE #####
type(data.columns)
#### END CODE ####

pandas.core.indexes.base.Index

Question 3: Describe your dataset

In [6]:
##### WRITE YOUR CODE HERE #####
data.describe()
#### END CODE ####

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Question 4: write a function that scale your features.
`Hint:` $x'= \dfrac{x-\mu}{\sigma}$ 
where $\mu$ is the mean and $\sigma$ the standard deviation of $x$.

In [7]:
##### WRITE YOUR CODE HERE #####
def feature_standard(data):
  ######## write your code #########
  dmean=np.mean(data, axis=0)
  dstd= np.mean(data, axis=0)
  data_Scaled= (data - dmean)/dstd


  return data_Scaled

#### END CODE ####

Question 5: Scale your features, except the target. 

In [8]:
##### WRITE YOUR CODE HERE #####
Standard_data=feature_standard(data)
data.iloc[:, 0:-1]=Standard_data.iloc[:, 0:-1]
data.head()

#### END CODE #### 

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.560447,0.224208,0.041886,0.704286,-1.000000,0.050244,0.328738,0.504172,1
1,-0.739925,-0.296908,-0.044938,0.412123,-1.000000,-0.168557,-0.256161,-0.067414,0
2,1.080596,0.513716,-0.073879,-1.000000,-1.000000,-0.271706,0.424102,-0.037330,1
3,-0.739925,-0.263821,-0.044938,0.119959,0.177953,-0.121671,-0.646094,-0.368248,0
4,-1.000000,0.133219,-0.421175,0.704286,1.105277,0.347187,3.848728,-0.007247,1


Question 6: Write a function that split your (x,y) to (x_train, y_train) and (x_test, y_test).
`Note:` This function must return an arrays.

In [9]:
##### WRITE YOUR CODE HERE #####
def split_data(df, train_percent= 0.8):
  np.random.seed(2)
  perm = np.random.permutation(df.index)

  n= len(df)
  train_index = int(train_percent * n)

  train = df.iloc[perm[:train_index]]
  test = df.iloc[perm[train_index:]]

  x_train, x_test, y_train, y_test= train.iloc[:, :-1], test.iloc[:, :-1], train.iloc[:, -1], test.iloc[:, -1]
  return x_train.values, x_test.values, y_train.values, y_test.values
#### END CODE ####

In [10]:
x_train, x_test, y_train, y_test= split_data(data) 
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((614, 8), (614,), (154, 8), (154,))

# Part 2: Logistic Regression without Regularization.

## Recaps:
1. Logistic/sigmoid function of $z$:
\begin{equation}
g(z)= \dfrac{1}{1+ exp(-z)}
\end{equation}
where $z= x^t \theta$ and $x, y \in \mathbb{R}^d$.
2. Derivative of Logistic/sigmoid function with respective to $z$:
\begin{equation}
dg(z)= g(z)(1-g(z))
\end{equation}
3. Cross-entropy loss:
\begin{equation}
l(\theta)= -\sum_{i= 1}^{N} \left(y_{true} \times \log y_{pred} + (1-y_{true}) \times \log (1-y_{pred}) \right)
\end{equation}
where $y_{true}= g(z)$, $z= x^t \theta$ and $y_{true}$ is the ground-truth for a given $x$.

4. Derivative of Cross-entropy loss with respective to $\theta$:
\begin{equation}
dl(\theta)= -\sum_{i= 1}^{N} x^t\left(y_{true} -y_{ped} \right)
\end{equation}
5. Apply Batch gradient descent to update $\theta$.

Question: Create a class called LogReg. See the description in the below cells.


In [41]:
class logistic_regression:
  def __init__(self, num_iters= 100, threshold= 0.5, tolerance= 1e-10, lr= 0.00001):
    ##### WRITE YOUR CODE HERE #####
    self.num_iters=num_iters
    self.threshold=threshold
    self.lr=lr
    self.tolerance=tolerance
    self.theta=None

    self.cost_history=[]
    self.cost_history_test=[]
    #### END CODE ####

  def add_ones(self, x):
    ##### WRITE YOUR CODE HERE #####
    x_new=np.hstack([np.ones((x.shape[0], 1)), x])
    return x_new
    #### END CODE ####

  def sigmoid(self, x):
    ##### WRITE YOUR CODE HERE #####
    h= x@self.theta
    #print(z)
    return 1/(1+np.exp(-h))
    #### END CODE ####

  def cross_entropy(self, x, y_true):
    ##### WRITE YOUR CODE HERE #####
    n = len(x) # Length of x
    y_pred = self.sigmoid(x)
    Cost = -((y_true.T@np.log(y_pred))+(1-y_true).T@np.log(1-y_pred))/n    
    return Cost
    #### END CODE ####

  def fit(self, x,y):
    ##### WRITE YOUR CODE HERE #####
    x= self.add_ones(x) # Add ones to x
    y= y.reshape(-1,1) # reshape y. This is optional, do it if needed
    self.theta= np.zeros((x.shape[1],1)) # Initialize theta to zeros vector >>> (x.shape[1])
    current_iter= 1
    norm= 1

    while (norm >= self.tolerance and current_iter < self.num_iters):
      theta_old = self.theta.copy() # Get old theta

      # make predictions
      y_pred= self.sigmoid(x)# using sigmoid function 

      # Gradient of cross-entropy
      grad= -x.T@(y - y_pred)
      grad= grad.reshape(-1,1) # Reshape, if it is needed

      # update rules
      theta_new= theta_old-self.lr*grad
      self.theta=theta_new
      # Compute the training loss
      self.cost_history.append(self.cross_entropy(x, y))
      self.cost_history_test.append(self.cross_entropy(self.add_ones(x_test), y_test))

      # Convergence criteria:
      if current_iter%100 == 0:
        print(f'cost for {current_iter} iteration : {self.cross_entropy(x, y)}')
      norm = np.linalg.norm(theta_old - self.theta)
      current_iter += 1
    #### END CODE ####
  
  def predict(self, x):
    proba= self.sigmoid(x)# Get probability of x
    result= [1 if i > 0.5 else 0 for i in proba]  # Convert proba to 0 or 1. hint: list comprehension
    return result
  
    
  def predict_proba(self, x):
    x= self.sigmoid() # Apply add ones to x
    y_pred_prob= None # Predict proba with sigmoid
    return None
  
  # def plot(self):
    ##### WRITE YOUR CODE HERE #####
    # plt.plot(cost_history, label='Train Loss')
    # plt.plot(cost_history_test, label='Test loss')

In [42]:
# Create a model by instanciating the class logistic_regression
model = logistic_regression(num_iters=10000)

In [43]:
%%time
model.fit(x_train, y_train)

cost for 100 iteration : [[0.66865321]]
cost for 200 iteration : [[0.65026448]]
cost for 300 iteration : [[0.63615174]]
cost for 400 iteration : [[0.62509761]]
cost for 500 iteration : [[0.61627359]]
cost for 600 iteration : [[0.60910308]]
cost for 700 iteration : [[0.60317669]]
cost for 800 iteration : [[0.5981987]]
cost for 900 iteration : [[0.59395225]]
cost for 1000 iteration : [[0.59027626]]
cost for 1100 iteration : [[0.58704976]]
cost for 1200 iteration : [[0.58418096]]
cost for 1300 iteration : [[0.5815996]]
cost for 1400 iteration : [[0.57925144]]
cost for 1500 iteration : [[0.5770943]]
cost for 1600 iteration : [[0.57509509]]
cost for 1700 iteration : [[0.57322774]]
cost for 1800 iteration : [[0.57147151]]
cost for 1900 iteration : [[0.56980982]]
cost for 2000 iteration : [[0.56822933]]
cost for 2100 iteration : [[0.56671925]]
cost for 2200 iteration : [[0.56527077]]
cost for 2300 iteration : [[0.56387668]]
cost for 2400 iteration : [[0.56253104]]
cost for 2500 iteration : [[

In [39]:
pred= model.predict(x_test)

ValueError: ignored

In [ ]:
pred

In [ ]:
# Predict Proba
print(model.predict_proba(x_test))

In [ ]:
model.plot()

## Push your works to Githup Repo

Setps:
1. Fork the repo: `https://github.com/AMMI-2022/test-live-coding`
2. Clone in your colab;
3. Push
4. Make a pull request

Set your credential

In [ ]:
my_token= ""
my_repository= ""
username= ""
useremail= ""

Clone the repo

In [ ]:
! git clone https://github.com/{username}/{my_repository}.git

In [ ]:
! git remote -v

Copy data to test-live-coding folder

In [ ]:
!cp diabetes.csv /content/test-live-coding

In [ ]:
%cd /content/test-live-coding

In [ ]:
!git status

In [ ]:
!git config user.email useremail
!git config user.name username

In [ ]:
!git add .
!git commit -m "Add my files"

In [ ]:
!git status

In [ ]:
!git push -u origin master --force

# If you cannot push, use:

In [ ]:
!git remote rm origin

In [ ]:
!git remote add origin https://{my_token}@github.com/{username}/{my_repository}.git
!git remote -v

In [ ]:
!git push -u origin master --force

---



---



---

# Assignment

Just follow the class logistic_regression, try to implement Logistic regression with `l1`, `l2` and `l1-l2`. That we call respectively `Lasso (least absolute shrinkage and selection operator)`, `Ridge` and `ElasticNet`.
Given cross-entropy:
\begin{equation}
  l(θ)= -\sum_{i= 1}^{N} \left(y_{true} \times \log y_{pred} + (1-y_{true}) \times \log (1-y_{pred}) \right)
\end{equation}

1. l1-regularization (LASSO):

\begin{equation}
   l(θ) + λ\| \theta\|_1
\end{equation}


2. l2-regularization (Ridge):
\begin{equation}
   l(θ)+ λ\| \theta\|_2
\end{equation}

3. l1-l2-regularization (ElasticNet):
\begin{equation}
  l(θ) + λ_1\| \theta\|_1 + λ_2\| \theta\|_2
\end{equation}

`Note: please use mini-batch as learning algorithm`

In [ ]:
class logistic_regression_l1:
  pass

In [ ]:
class logistic_regression_l2:
  pass

In [ ]:
class logistic_regression_l1_l2:
  pass

# Concepts [Assignment]:
1. Why there is no close form for cross-entropy loss?
2. Why we need cross-entropy loss instead of mse?
3. Why is logistic regression a type of classification technique and not a regression?